**TODO** select non-overlapping windows only (to avoid the non-independence between sites)

The basic idea is to find out which categories of genomic regions (exons, promoters, enhancers, etc) contribute the most to the Nea. ancestry observed in present day Europeans. For example: have coding regions or enhancers been the main drivers of selection against Nea. introgression?

Links:

* info about different coordinate types: https://groups.google.com/forum/#!msg/biomart-users/OtQbAx3y9CA/wrF19ID1AgAJ
* https://www.biostars.org/p/2005/
* http://www.bioconductor.org/packages/release/bioc/vignettes/biomaRt/inst/doc/biomaRt.html#attribute-pages
* http://www.ensembl.org/info/data/biomart/biomart_r_package.html
* http://www.ensembl.info/blog/2015/06/01/biomart-or-how-to-access-the-ensembl-data-from-r/
* for checking with manually downloaded data: http://www.ensembl.org/info/data/biomart/how_to_use_biomart.html
* biotypes FAQ: http://www.ensembl.org/Help/Glossary

Following [this](http://www.ensembl.org/info/data/biomart/biomart_r_package.html) tutorial, I want to extract coordinates of exonic and regulatory regions from the Ensembl database and then calculate the density of such regions in a defined window around each SNP.

These densities will they be used as predictors in a linear model, predicting the Nea. ancestry at each site.

Alternatively, I could just test if the distribution of densities for different regions differ based on frequency of Nea. alleles at each site.

In [1]:
suppressWarnings({
    library(biomaRt)
    library(rtracklayer)
    library(BSgenome.Hsapiens.UCSC.hg19)
})

Loading required package: GenomicRanges
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit

Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The following object

In [2]:
suppressWarnings({
    library(tidyverse)
    library(stringr)
    library(magrittr)
})

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
collapse(): dplyr, Biostrings, IRanges
combine():  dplyr, BiocGenerics
compact():  purrr, XVector
desc():     dplyr, IRanges
expand():   tidyr, S4Vectors
filter():   dplyr, stats
first():    dplyr, S4Vectors
lag():      dplyr, stats
Position(): ggplot2, BiocGenerics, base
reduce():   purrr, GenomicRanges, IRanges
rename():   dplyr, S4Vectors
select():   dplyr, biomaRt
simplify(): purrr, IRanges
slice():    dplyr, XVector, IRanges

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



## Get coordinates of admixture array sites

In [3]:
source("../R/utils.R")

In [4]:
nonafr_info <- load_sgdp_info("../raw_data/10_24_2014_SGDP_metainformation_update.txt") %>% 
    filter(! Region %in% c("Africa", "Oceania"))

Warning message:
“Missing column names filled in: 'X12' [12], 'X13' [13], 'X14' [14], 'X15' [15], 'X16' [16], 'X17' [17], 'X18' [18], 'X19' [19], 'X20' [20], 'X21' [21], 'X22' [22], 'X23' [23], 'X24' [24]”Parsed with column specification:
cols(
  .default = col_character(),
  Latitude = col_double(),
  Longitude = col_double(),
  Coverage = col_integer(),
  HetRateAuto = col_double()
)
See spec(...) for full column specifications.


In [5]:
genotypes <-
    load_dataset("../clean_data/ice_age.tsv", "../clean_data/sgdp.tsv", "../clean_data/archaics.tsv",
                 random_sample=FALSE) %>%
    select(c("chrom", "pos", nonafr_info$name)) %>%
    mutate(start=pos, end=pos) %>%
    select(-pos) %>%
    makeGRangesFromDataFrame(keep.extra.columns=TRUE)

seqlevelsStyle(genotypes) <- "UCSC"
seqinfo(genotypes) <- seqinfo(BSgenome.Hsapiens.UCSC.hg19)

Parsed with column specification:
cols(
  .default = col_integer(),
  ref = col_character(),
  alt = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_integer(),
  ref = col_character(),
  alt = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  chrom = col_integer(),
  pos = col_integer(),
  ref = col_character(),
  alt = col_character(),
  Altai = col_integer(),
  Vindija = col_integer(),
  Denisovan = col_integer()
)
Joining, by = c("chrom", "pos", "ref", "alt")
Joining, by = c("chrom", "pos", "ref", "alt")


In [6]:
sites <- granges(genotypes)

In [7]:
sites

GRanges object with 484016 ranges and 0 metadata columns:
           seqnames               ranges strand
              <Rle>            <IRanges>  <Rle>
       [1]     chr1     [847983, 847983]      *
       [2]     chr1     [853089, 853089]      *
       [3]     chr1     [853596, 853596]      *
       [4]     chr1     [854793, 854793]      *
       [5]     chr1     [867552, 867552]      *
       ...      ...                  ...    ...
  [484012]    chr22 [51142979, 51142979]      *
  [484013]    chr22 [51145136, 51145136]      *
  [484014]    chr22 [51154908, 51154908]      *
  [484015]    chr22 [51174091, 51174091]      *
  [484016]    chr22 [51177248, 51177248]      *
  -------
  seqinfo: 93 sequences (1 circular) from hg19 genome

# Fetch coordinates of different genomic regions

In [8]:
regions <- list()

## Exon coordinates

Show all the available biomarts for hg19:

In [9]:
listMarts(host="grch37.ensembl.org")

biomart,version
ENSEMBL_MART_ENSEMBL,Ensembl Genes 89
ENSEMBL_MART_SNP,Ensembl Variation 89
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 89


Connect to the human gene Ensembl dataset:

In [10]:
ensembl_mart_genes <- useMart("ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org")
listDatasets(ensembl_mart_genes) %>% filter(str_detect(dataset, "sapiens"))

dataset,description,version
hsapiens_gene_ensembl,Human genes (GRCh37.p13),GRCh37.p13


In [11]:
genes <- useDataset(dataset="hsapiens_gene_ensembl", mart=ensembl_mart_genes)

Get coordinates of protein_coding regions from the gene annotation database:

In [12]:
regions[["protein_coding"]] <-
    getBM((listAttributes(genes) %>% filter(page=="structure"))$name,    # get all attributes
          filters=c("chromosome_name", "biotype"),                       # filter for chromosome and biotype
          values=list(1:22, "protein_coding"),                           # chromosomes 1:22 and "protein_coding"
          mart=genes) %>%
    dplyr::select(chromosome_name, exon_chrom_start, exon_chrom_end) %>%
    setNames(c("chrom", "start", "end")) %>%
    arrange(chrom, start) %>%
    makeGRangesFromDataFrame

## Coordinates of regulatory features

It took forever to find out, how to get the regulatory features. In the end I found it totally by accident in some bugreport mail :( https://support.bioconductor.org/p/39545/

Show all the available biomarts for hg19:

In [13]:
listMarts(host="grch37.ensembl.org")

biomart,version
ENSEMBL_MART_ENSEMBL,Ensembl Genes 89
ENSEMBL_MART_SNP,Ensembl Variation 89
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 89


Connect to the human gene Ensembl dataset:

In [14]:
ensembl_mart_funcgen <- useMart("ENSEMBL_MART_FUNCGEN", host="grch37.ensembl.org")
listDatasets(ensembl_mart_funcgen) %>% filter(str_detect(dataset, "sapiens"))

dataset,description,version
hsapiens_regulatory_feature,Human Regulatory Features (GRCh37.p13),GRCh37.p13
hsapiens_external_feature,Human Other Regulatory Regions (GRCh37.p13),GRCh37.p13
hsapiens_motif_feature,Human Binding Motifs (GRCh37.p13),GRCh37.p13
hsapiens_mirna_target_feature,Human miRNA Target Regions (GRCh37.p13),GRCh37.p13
hsapiens_annotated_feature,Human Regulatory Evidence (GRCh37.p13),GRCh37.p13


In [15]:
regulation <- useDataset(dataset="hsapiens_regulatory_feature", mart=ensembl_mart_funcgen)

Download the dataframe from Biomart:

In [16]:
regulatory_features_df <-
    getBM(attributes=c("chromosome_name", "chromosome_start", "chromosome_end", "feature_type_name"),
          filters="chromosome_name",
          values=1:22,
          mart=regulation) %>%
    setNames(c("chrom", "start", "end", "feature")) %>%  # rename the columns to BED names
    mutate(feature=str_replace_all(tolower(feature), " ", "_"))

In [17]:
table(regulatory_features_df$feature)


       ctcf_binding_site                 enhancer           open_chromatin 
                   55194                    27577                   107233 
                promoter promoter_flanking_region          tf_binding_site 
                   14287                    45496                    22376 

In [18]:
regions <- c(regions, lapply(split(regulatory_features_df, regulatory_features_df$feature),
                             makeGRangesFromDataFrame))

## Coordinates of primate phastCons elements

How to retrieve them: https://support.bioconductor.org/p/25587/

Per-base vs elements diference: https://www.biostars.org/p/2129/#2143

In [19]:
library(rtracklayer)

In [20]:
session <- browserSession()
genome(session) <- "hg19"

In [21]:
query <- ucscTableQuery(session, "cons46way", GRangesForUCSCGenome("hg19", chrom=paste0("chr", 1:22)))

In [22]:
tableNames(query)

[1] "phyloP46wayPrimates"             "phyloP46wayPlacental"           
 [3] "phyloP46wayAll"                  "phastCons46wayPrimates"         
 [5] "phastCons46wayPlacental"         "phastCons46way"                 
 [7] "phastConsElements46wayPrimates"  "phastConsElements46wayPlacental"
 [9] "phastConsElements46way"          "multiz46way"

In [23]:
tableName(query) <- "phastConsElements46wayPrimates"

In [24]:
regions[["priPhastCons"]] <-
    getTable(query) %>%
    select(-bin, -name, -score) %>%
    makeGRangesFromDataFrame(starts.in.df.are.0based=TRUE)

## Reduce all regions and convert their coordinates to UCSC hg19

Convert all coordinate dataframes into individual `GRanges` objects:

In [25]:
regions <- lapply(regions,
                  function(r) {
                      reduced_r <- IRanges::reduce(r)
                      seqlevelsStyle(reduced_r) <- "UCSC"
                      seqinfo(reduced_r) <- seqinfo(BSgenome.Hsapiens.UCSC.hg19)
                      
                      reduced_r
                  })

In [26]:
names(regions)

[1] "protein_coding"           "ctcf_binding_site"       
[3] "enhancer"                 "open_chromatin"          
[5] "promoter"                 "promoter_flanking_region"
[7] "tf_binding_site"          "priPhastCons"

## Get the hits of each site in different genomic regions

In [27]:
for (r in names(regions)) {
    col_name <- paste0("hits_", r)

    # find which sites are falling within a given region
    hits <- findOverlaps(sites, regions[[r]])

    # mark which sites hit a region
    mcols(sites)[[col_name]] <- 0
    mcols(sites[queryHits(hits)])[[col_name]] <- 1
}

## Get average primate phastCons per window

In [28]:
priPhastCons_avg <-
    read.table("../clean_data/annotations/priPhCons__100000bp.bed", header=FALSE) %>%
    .[1:4] %>%
    setNames(c("chrom", "start", "end", "value")) %>%
    arrange(chrom, start) %>% 
    mutate(chrom=paste0("chr", chrom)) %>% 
    makeGRangesFromDataFrame(starts.in.df.are.0based=TRUE, keep.extra=TRUE)

In [29]:
mcols(sites)[["priPhastCons_avg"]] <- subsetByOverlaps(priPhastCons_avg, sites)$value

## Add B values here

In [30]:
source("../R/assign_bvals.R")

Warning message:
“package ‘BSgenome.Hsapiens.UCSC.hg18’ was built under R version 3.4.0”
Attaching package: ‘BSgenome.Hsapiens.UCSC.hg18’

The following object is masked from ‘package:BSgenome.Hsapiens.UCSC.hg19’:

    Hsapiens



In [31]:
sites <- assign_bvals(sites, bval_path="../raw_data/bkgd/", chain_path="../raw_data/hg18ToHg19.over.chain")

## Calculate Neanderthal allele frequency at each locus

A couple of sanity checks - do we get an expected average Nea. allele frequency between 0 and 1 and mean of around 1.8% in Europeans?

In [32]:
summary(sites$freq_eur)
summary(sites$freq_all)

Length  Class   Mode 
     0   NULL   NULL 

Length  Class   Mode 
     0   NULL   NULL 

## Convert the final `GRanges` object to a normal data frame for the analyses bellow

In [144]:
tbl <- as.data.frame(sites) %>%
    select(-width, -strand) %>%
    rename(chrom=seqnames)

In [145]:
head(tbl)

chrom,start,end,freq_eur,freq_all,hits_protein_coding,hits_ctcf_binding_site,hits_enhancer,hits_open_chromatin,hits_promoter,hits_promoter_flanking_region,hits_tf_binding_site,hits_priPhastCons,priPhastCons_avg
chr1,847983,847983,0,0.005076142,0,1,0,0,0,0,0,0,0.047667
chr1,853089,853089,0,0.005102041,0,1,0,0,0,0,0,0,0.049807
chr1,853596,853596,0,0.005076142,0,0,0,0,0,0,0,0,0.049866
chr1,854793,854793,0,0.005076142,0,0,0,0,0,0,0,0,0.049938
chr1,867552,867552,0,0.005076142,0,1,0,0,0,0,0,1,0.047507
chr1,871401,871401,0,0.007614213,0,0,0,1,0,0,0,0,0.045560


In [146]:
save(genotypes, regions, sites, tbl,
     file="../RData/predictor_analysis.RData")

## Modeling the dependence of Nea. frequency on functional predictors

In [147]:
load("../RData/predictor_analysis.RData")

In [169]:
fit <- glm(data=tbl, freq_all > 0 ~ hits_protein_coding + hits_priPhastCons + hits_promoter + hits_promoter_flanking_region + hits_ctcf_binding_site + hits_enhancer + hits_open_chromatin + hits_tf_binding_site, family="binomial")
summary(fit)


Call:
glm(formula = freq_all > 0 ~ hits_protein_coding + hits_priPhastCons + 
    hits_promoter + hits_promoter_flanking_region + hits_ctcf_binding_site + 
    hits_enhancer + hits_open_chromatin + hits_tf_binding_site, 
    family = "binomial", data = tbl)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.234  -1.192   1.156   1.163   1.320  

Coefficients:
                               Estimate Std. Error z value             Pr(>|z|)
(Intercept)                    0.033716   0.003072  10.974 < 0.0000000000000002
hits_protein_coding           -0.070393   0.016332  -4.310     0.00001631191334
hits_priPhastCons             -0.111258   0.015716  -7.079     0.00000000000145
hits_promoter                 -0.181068   0.028410  -6.373     0.00000000018502
hits_promoter_flanking_region -0.065357   0.017707  -3.691             0.000223
hits_ctcf_binding_site         0.015843   0.022013   0.720             0.471715
hits_enhancer                 -0.007648   0.031692  -0.241    

In [167]:
fit <- glm(data=sample_frac(tbl, replace=TRUE), freq_all > 0 ~ hits_protein_coding + hits_priPhastCons + hits_promoter + hits_promoter_flanking_region + hits_ctcf_binding_site + hits_enhancer + hits_open_chromatin + hits_tf_binding_site, family="binomial")
summary(fit)


Call:
glm(formula = freq_all > 0 ~ hits_protein_coding + hits_priPhastCons + 
    hits_promoter + hits_promoter_flanking_region + hits_ctcf_binding_site + 
    hits_enhancer + hits_open_chromatin + hits_tf_binding_site, 
    family = "binomial", data = sample_frac(tbl, replace = TRUE))

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.240  -1.193   1.162   1.162   1.341  

Coefficients:
                               Estimate Std. Error z value             Pr(>|z|)
(Intercept)                    0.035509   0.003073  11.554 < 0.0000000000000002
hits_protein_coding           -0.043933   0.016371  -2.684              0.00728
hits_priPhastCons             -0.116891   0.015723  -7.434    0.000000000000105
hits_promoter                 -0.242634   0.028221  -8.598 < 0.0000000000000002
hits_promoter_flanking_region -0.079504   0.017620  -4.512    0.000006418662924
hits_ctcf_binding_site        -0.008874   0.022027  -0.403              0.68704
hits_enhancer                 -0.